In [1]:
import cv2
import numpy as np

In [2]:
video_path = "data/Data/kaniula1.avi"
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error: Could not open the video file.")
    exit()

ret, first_frame = cap.read()

if not ret:
    print("Error: Could not read the first frame.")
    cap.release()
    exit()

cap.release()

cv2.imshow("First Frame", first_frame)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [3]:
def create_binary_image(input_frame, box_size=5, offset=10):
    gray_image = cv2.cvtColor(input_frame, cv2.COLOR_BGR2GRAY)

    binary_image = np.zeros_like(gray_image)

    for i in range(box_size, gray_image.shape[0] - box_size):
        for j in range(box_size, gray_image.shape[1] - box_size):
            box_mean = np.mean(gray_image[i - box_size : i + box_size + 1, j - box_size : j + box_size + 1])
            if gray_image[i, j] > box_mean + offset:
                binary_image[i, j] = 255

    # cv2.imshow("Binary Image", binary_image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

    return binary_image



In [24]:
#Zastosowanie maski

def apply_mask(input_array, border):
    height, width = input_array.shape

    mask = np.zeros((height, width), dtype=np.uint8)

    center_x = width // 2
    center_y = height // 2

    for i in range(-100, 100):
        mask[center_y + i, center_x + i:] = 255

    mask[:, :border] = 0
    mask[:, -border:] = 0

    result_array = cv2.bitwise_and(input_array, input_array, mask=mask)
    
    # cv2.imshow("Result Image", result_array)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    
    return result_array

In [5]:
#Przefiltrowanie plam które są za małe
def filter_blobs(input_array, area_threshold=50, aspect_ratio_threshold=1.0):
    contours, _ = cv2.findContours(input_array, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    mask = np.zeros_like(input_array)

    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)

        aspect_ratio = float(w) / h

        if cv2.contourArea(contour) >= area_threshold and aspect_ratio <= aspect_ratio_threshold:
            cv2.drawContours(mask, [contour], 0, 255, thickness=cv2.FILLED)

    result_array_filtered = cv2.bitwise_and(input_array, mask)

    # cv2.imshow('Filtered Result', result_array_filtered)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

    return result_array_filtered

In [6]:
# Wypełnienie dziur w plamach

def fill_blobs(input_array):
    input_array = input_array.astype(np.uint8)

    contours, hierarchy = cv2.findContours(input_array, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for contour in contours:
        cv2.drawContours(input_array, [contour], 0, 255, thickness=cv2.FILLED)
        
    # cv2.imshow('Filtered Result', result_array_filtered)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()
    return input_array

In [8]:
def detect_rightmost_blobs(binary_image, color_image, rect_width=20, rect_height=50, num_blobs=2):
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        return None, None

    # Sort contours based on their x-coordinate
    contours = sorted(contours, key=lambda c: cv2.boundingRect(c)[0])

    rightmost_contours = contours[-num_blobs:]  # Get the rightmost N contours
    centroid_positions = []

    for contour in rightmost_contours:
        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(color_image, (x, y), (x + rect_width, y + rect_height), (0, 255, 0), 2)

        centroid_x = x + w // 2
        centroid_y = y + h // 2

        centroid_positions.append((centroid_x, centroid_y))

    return color_image, centroid_positions

In [9]:
import math

def euc_distance(point1, point2):
    x1, y1 = point1
    x2, y2 = point2
    distance = math.sqrt((x2 - x1)**2 + (y2 - y1)**2)
    return distance

In [17]:
def in_bounds(point1, point2):
    x1, y1 = point1
    x2, y2 = point2
    distance_hor = abs(x1-x2)
    distance_ver = abs(y1-y2)

    return distance_hor < 6 and distance_ver < 30

In [30]:
video_path = 'data/Data/kaniula1.avi'

cap = cv2.VideoCapture(video_path)

previous_centroid = (0.0, 0.0)
shift = 0
radius = 10

frame_counter = 0

while True:
    ret, frame = cap.read()

    if not ret:
        break

    frame_counter += 1

    binary_image = create_binary_image(frame)
    masked_image = apply_mask(binary_image, 125)
    filtered_image = filter_blobs(masked_image)
    filled_image = fill_blobs(filtered_image)
    result_image, centroid_positions = detect_rightmost_blobs(filled_image, frame)

    if(in_bounds(previous_centroid, centroid_positions[0])):
        print("Tracking the blob, frame number: ", frame_counter)
    elif(in_bounds(previous_centroid, centroid_positions[1])):
        shift -= 1
        print("Previous blob disappeared, shift = ", shift)
    elif(not in_bounds(previous_centroid, centroid_positions[0]) and euc_distance(centroid_positions[0], previous_centroid) < 100):
        shift += 1
        print("New rightmost blob detected, shift = ", shift)
        
    previous_centroid = centroid_positions[0]

    cv2.imshow("Binary Image", filled_image)
    cv2.imshow("Result Image", frame)
    cv2.waitKey(1)

Tracking the blob, frame number:  2
Tracking the blob, frame number:  3
Tracking the blob, frame number:  4
Tracking the blob, frame number:  5
Tracking the blob, frame number:  6
Tracking the blob, frame number:  7
Tracking the blob, frame number:  8
Tracking the blob, frame number:  9
Tracking the blob, frame number:  10
Tracking the blob, frame number:  11
Tracking the blob, frame number:  12
Tracking the blob, frame number:  13
Tracking the blob, frame number:  14
Tracking the blob, frame number:  15
Tracking the blob, frame number:  16
Tracking the blob, frame number:  17
Tracking the blob, frame number:  18
Tracking the blob, frame number:  19
Tracking the blob, frame number:  20
Tracking the blob, frame number:  21
Tracking the blob, frame number:  22
Tracking the blob, frame number:  23
Tracking the blob, frame number:  24
Tracking the blob, frame number:  25
Tracking the blob, frame number:  26
Tracking the blob, frame number:  27
Tracking the blob, frame number:  28
Tracking 

TypeError: 'NoneType' object is not subscriptable

In [27]:
video_path = 'data/Data/kaniula2.avi'

cap = cv2.VideoCapture(video_path)

previous_centroid = (0.0, 0.0)
shift = 0
radius = 10

frame_counter = 0

while True:
    ret, frame = cap.read()

    if not ret:
        break

    frame_counter += 1

    binary_image = create_binary_image(frame)
    masked_image = apply_mask(binary_image, 125)
    filtered_image = filter_blobs(masked_image)
    filled_image = fill_blobs(filtered_image)
    result_image, centroid_positions = detect_rightmost_blobs(filled_image, frame)

    if(in_bounds(previous_centroid, centroid_positions[0])):
        print("Tracking the blob, frame number: ", frame_counter)
    elif(in_bounds(previous_centroid, centroid_positions[1])):
        shift -= 1
        print("Previous blob disappeared, shift = ", shift)
    elif(not in_bounds(previous_centroid, centroid_positions[0]) and euc_distance(centroid_positions[0], previous_centroid) < 100):
        shift += 1
        print("New rightmost blob detected, shift = ", shift)
        
    previous_centroid = centroid_positions[0]

    cv2.imshow("Binary Image", filled_image)
    cv2.imshow("Result Image", frame)
    cv2.waitKey(1)

Tracking the blob, frame number:  2
Tracking the blob, frame number:  3
Tracking the blob, frame number:  4
Tracking the blob, frame number:  5
Tracking the blob, frame number:  6
Tracking the blob, frame number:  7
Tracking the blob, frame number:  8
Tracking the blob, frame number:  9
Tracking the blob, frame number:  10
Tracking the blob, frame number:  11
Tracking the blob, frame number:  12
Tracking the blob, frame number:  13
Tracking the blob, frame number:  14
Tracking the blob, frame number:  15
Tracking the blob, frame number:  16
Tracking the blob, frame number:  17
Tracking the blob, frame number:  18
Tracking the blob, frame number:  19
Tracking the blob, frame number:  20
Previous blob disappeared, shift =  -1
Tracking the blob, frame number:  22
Tracking the blob, frame number:  23
Tracking the blob, frame number:  24
Tracking the blob, frame number:  25
Tracking the blob, frame number:  26
Tracking the blob, frame number:  27
Tracking the blob, frame number:  28
Trackin

In [31]:
video_path = 'data/Data/kaniula3.avi'

cap = cv2.VideoCapture(video_path)

previous_centroid = (0.0, 0.0)
shift = 0
radius = 10

frame_counter = 0

while True:
    ret, frame = cap.read()

    if not ret:
        break

    frame_counter += 1

    binary_image = create_binary_image(frame)
    masked_image = apply_mask(binary_image, 125)
    filtered_image = filter_blobs(masked_image)
    filled_image = fill_blobs(filtered_image)
    result_image, centroid_positions = detect_rightmost_blobs(filled_image, frame)

    if(in_bounds(previous_centroid, centroid_positions[0])):
        print("Tracking the blob, frame number: ", frame_counter)
    elif(in_bounds(previous_centroid, centroid_positions[1])):
        shift -= 1
        print("Previous blob disappeared, shift = ", shift)
    elif(not in_bounds(previous_centroid, centroid_positions[0]) and euc_distance(centroid_positions[0], previous_centroid) < 100):
        shift += 1
        print("New rightmost blob detected, shift = ", shift)
        
    previous_centroid = centroid_positions[0]

    cv2.imshow("Binary Image", filled_image)
    cv2.imshow("Result Image", frame)
    cv2.waitKey(1)

Tracking the blob, frame number:  2
Tracking the blob, frame number:  3
Tracking the blob, frame number:  4
Tracking the blob, frame number:  5
Tracking the blob, frame number:  6
Tracking the blob, frame number:  7
Tracking the blob, frame number:  8
Tracking the blob, frame number:  9
Tracking the blob, frame number:  10
Tracking the blob, frame number:  11
Tracking the blob, frame number:  12
Tracking the blob, frame number:  13
Tracking the blob, frame number:  14
Tracking the blob, frame number:  15
Tracking the blob, frame number:  16
Tracking the blob, frame number:  17
Tracking the blob, frame number:  18
Tracking the blob, frame number:  19
Tracking the blob, frame number:  20
Tracking the blob, frame number:  21
Tracking the blob, frame number:  22
Tracking the blob, frame number:  23
Tracking the blob, frame number:  24
Tracking the blob, frame number:  25
Tracking the blob, frame number:  26
Tracking the blob, frame number:  27
Tracking the blob, frame number:  28
Tracking 

In [33]:
def calculate_difference(real_shift, calculated_shift):
    return abs(real_shift - calculated_shift)

data = [
    {"movie": 1, "real_shift": 61, "calculated_shift": 92},
    {"movie": 2, "real_shift": 7, "calculated_shift": -23},
    {"movie": 3, "real_shift": 38, "calculated_shift": -16},
]

print("movie | real shift | calculated shift | DIFF")

for row in data:
    movie = row["movie"]
    real_shift = row["real_shift"]
    calculated_shift = row["calculated_shift"]
    diff = calculate_difference(real_shift, calculated_shift)
    
    # Print the row values
    print(f"{movie}\t|\t{real_shift} - {calculated_shift} = {real_shift + calculated_shift}\t|\t{calculated_shift}\t|\t{diff}")

movie | real shift | calculated shift | DIFF
1	|	61 - 92 = 153	|	92	|	31
2	|	7 - -23 = -16	|	-23	|	30
3	|	38 - -16 = 22	|	-16	|	54


In [34]:
#pip install prettytable

Note: you may need to restart the kernel to use updated packages.


In [38]:
from prettytable import PrettyTable

data = [
    {"movie": 1, "real_shift": 61, "calculated_shift": 92},
    {"movie": 2, "real_shift": -27, "calculated_shift": -23},
    {"movie": 3, "real_shift": -21, "calculated_shift": -16},
]

table = PrettyTable()
table.field_names = ["movie", "real shift", "calculated shift", "DIFF"]

for row in data:
    movie = row["movie"]
    real_shift = row["real_shift"]
    calculated_shift = row["calculated_shift"]
    diff = abs(real_shift - calculated_shift)
    
    table.add_row([movie, f"{real_shift}", calculated_shift, diff])

print(table)

+-------+------------+------------------+------+
| movie | real shift | calculated shift | DIFF |
+-------+------------+------------------+------+
|   1   |     61     |        92        |  31  |
|   2   |    -27     |       -23        |  4   |
|   3   |    -21     |       -16        |  5   |
+-------+------------+------------------+------+
